In [6]:
#os.listDir : liste tous les éléments à partir du chemin donnée
#os.path.join : concatenne les chemins


import os
import csv
import imageio
import numpy
import matplotlib.pyplot as plt
import torchvision as torch
import numpy as np
from PIL import Image
from posix import write
from tqdm import tqdm

In [36]:
# import archive.zip dans l'ordi
! 7z x /content/archive.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 728990397 bytes (696 MiB)

Extracting archive: /content/archive.zip
--
Path = /content/archive.zip
Type = zip
Physical Size = 728990397

  0%      6% 49 - Dataset/0/Colour/7.jpg                                 10% 80 - Dataset/11/Colour/34.jpg                                   13% 112 - Dataset/15/Colour/3.jpg                                   16% 132 - Dataset/16/Colour/15.jpg                                   

In [41]:
def initialiseData() :
  CHEMIN = "/content/Dataset/"
  try :
    fichierCSV = open("/content/fichier.csv",'w+',newline='',encoding='utf-8')
  except :
    print("erreur création fichier fichier.csv")
  csvVersion = csv.writer(fichierCSV)
  csvVersion.writerow(["image_normal","image_BW"])
  for volume in os.listdir(CHEMIN) :
    volumePath = os.path.join(CHEMIN,volume)
    cheminVolumeNoirBlanc = os.path.join(volumePath,"BlackAndWhiteImage")
    try :
      os.mkdir(cheminVolumeNoirBlanc)
    except  :
      print("erreur création dossier")
    imagesPath = os.listdir(os.path.join(volumePath,"Colour"))
    for imagePath in imagesPath :
      cheminImageGris = "BW_" + imagePath
      colorVolumePath = os.path.join(volumePath,"Colour",imagePath)
      blackWhiteVolumePath = os.path.join(cheminVolumeNoirBlanc,cheminImageGris)
      csvVersion.writerow([colorVolumePath,blackWhiteVolumePath])
      imageNormal = Image.open(colorVolumePath)
      imageBlackWhite = imageNormal.convert('L')
      imageBlackWhite.save(blackWhiteVolumePath)



In [42]:
initialiseData()

In [43]:
from torch.utils.data import Dataset
import torchvision
from torchvision.io import read_image, ImageReadMode
import pandas as pd

In [ ]:
class CustomDataSet(Dataset) :
  def __init__(self) :
    super.__init__()
    ''' 'attribut :
            cheminPourCsv
            taille Image
      '''
  def __len__(self) :
    return len('''taille Image''')
  def __getitem__(self,index) :
    '''
      récupérer les chemin du csv
      ouvrir les image en tant que tanser
      changer la taille avec l'attribut taille Image
      mettre entre -1 et 1 (opération : (rgb-127.5)/127.5)
    '''
    pass